
# PySpark Airline Flight Analysis

This notebook demonstrates how to work with **Flights dataset** using **PySpark RDDs**.  
We will explore how to load data from CSV files, filter headers, parse data, and perform basic analytics.

---

### **Objectives**
- Load CSV data into RDD using Spark.
- Remove header rows from datasets.
- Parse date, time, and numeric fields.
- Compute flight-level statistics using RDD transformations and actions.



## Initialize Spark Session and Load Data

We start by creating a `SparkSession` and reading the CSV file as text into RDDs.


In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


flightsDataPath = 'flights.csv'

flightsRdd = spark.sparkContext.textFile(flightsDataPath)




## Parsing Flight Data

We’ll now parse flight records to extract useful fields such as dates, times, and distances.  
To represent structured records, we use Python’s `namedtuple` along with datetime parsing.


In [ ]:


flightsRdd.first()
flightsRdd.count()


for i in flightsRdd.take(10): 
    print(i)


from datetime import datetime
from collections import namedtuple

fields = ('date', 'airline', 'flightnum', 'origin', 'dest', 
          'dep', 'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

Flight = namedtuple('Flight', fields, verbose=True)

DATE_FMT = '%Y-%m-%d'
TIME_FMT = '%H%M'

def parse(row):
    row[0] = datetime.strptime(row[0], DATE_FMT).date()
    row[5] = datetime.strptime(row[5], TIME_FMT).time()
    row[6] = float(row[6])
    row[7] = datetime.strptime(row[7], TIME_FMT).time()
    row[8] = float(row[8])
    row[9] = float(row[9])
    row[10] = float(row[10])
    return Flight(*row[:11])

flightsParsedRdd = flightsRdd.map(lambda x: x.split(",")).map(parse)
flightsParsedRdd.first()



## Flight-Level Analysis

We can perform aggregate computations such as calculating **average distance** and **percentage of delayed flights** using RDD actions like `reduce` and `filter`.


In [ ]:

# Average distance travelled by a flight
totalDistance = flightsParsedRdd.map(lambda x: x.distance).reduce(lambda x, y: x + y)
avgDistance = totalDistance / flightsParsedRdd.count()
avgDistance


In [ ]:

# Percentage of flights with delays
flightsParsedRdd.filter(lambda x: x.dep_delay > 0).count() / float(flightsParsedRdd.count())



## Summary

In this notebook, we learned how to:
- Load and preprocess CSV data using PySpark RDDs.  
- Parse structured records using `namedtuple` and `datetime`.  
- Perform analytical computations such as averages and ratios using RDD transformations and actions.

This demonstrates how Spark RDDs can handle **large-scale flight data** with ease and flexibility.
